In [ ]:
def build_stump(X,y,weights):
    n_samples,n_features=X.shape
    best_stump={}
    min_error=float('inf')
    for feature in range(n_features):
        feature_values=np.unique(X[:, feature])
        for threshold in feature_values:
            for inequality in ['lt','gt']:
                predictions=np.ones(n_samples)
                if inequality=='lt':
                    predictions[X[:,feature]<threshold]=-1
                else:
                    predictions[X[:,feature]>threshold]=-1
                error=sum(weights[predictions!=y])
                if error<min_error:
                    min_error=error
                    best_stump['feature']=feature
                    best_stump['threshold']=threshold
                    best_stump['inequality']=inequality
                    best_stump['error']=error
                    best_stump['predictions']=predictions
    return best_stump

def adaboost(X,y,n_estimators):
    n_samples=len(y)
    weights=np.ones(n_samples)/n_samples
    classifiers=[]
    for i in range(n_estimators):
        stump=build_stump(X,y,weights)
        error=stump['error']
        alpha=0.5*np.log((1-error)/max(error, 1e-10))
        stump['alpha']=alpha
        classifiers.append(stump)
        weights*=np.exp(-alpha*y*stump['predictions'])
        weights/=sum(weights)
    return classifiers

def adaboost_predict(classifiers, X):
    predictions=np.zeros(X.shape[0])
    for classifier in classifiers:
        stump_predictions=np.ones(X.shape[0])
        feature=classifier['feature']
        threshold=classifier['threshold']
        inequality=classifier['inequality']
        if inequality=='lt':
            stump_predictions[X[:,feature]<threshold]=-1
        else:
            stump_predictions[X[:,feature]>threshold]=-1

        predictions+=classifier['alpha']*stump_predictions
   
    return np.sign(predictions)

def accuracy(actual,predictions):
    correct=0.0
    for i in range(len(y_test)):
        correct+=(actual[i]==predictions[i])
    return correct/float(len(y_test))

y_test=np.where(y_test==1,1,-1)
y_train=np.where(y_train==1,1,-1)
n_estimators=20
classifiers=adaboost(X_train,y_train,n_estimators)
predictions=adaboost_predict(classifiers, X_test)
print(f"Accuracy is {accuracy(y_test,predictions)}")
